In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline
from IPython.display import YouTubeVideo

# ML import
from transformers import pipeline              # Pre-trained models
from sklearn.model_selection import train_test_split # Splitting the data set
from sklearn.preprocessing import MinMaxScaler       # Normalization
import torch                                   # PyTorch
import torch.nn as nn                          # PyTorch building blocks

# Neural Nets

<img src='https://imgs.xkcd.com/comics/trained_a_neural_net.png' width="270">

Source: [xqcd 2173](https://xkcd.com/2173/)

## Content

The goal of this walkthrough is to provide you with insights on Neural Nets. After presenting the main concepts, you will be introduced to the techniques to implement your own neural network in Python, with PyTorch and using a pre-build Neural Net. 

This notebook is organized as follows:
- [Background](#Background)
    - [Neurons](#Neurons)
        - [Biological Neuron](#Biological-Neuron)
        - [Artificial neuron and perceptron](#Artificial-neuron-and-perceptron)
            - [Activation function](#Activation-function)
    - [Some Neural Nets](#Some-Neural-Nets)
        - [Multilayer Perceptron (MLP)](#Multilayer-Perceptron-(MLP))
        - [Convolutional Neural Nerwork (CNN)](#Convolutional-Neural-Nerwork-(CNN))
        - [Recurrent Neural Network (RNN)](#Recurrent-Neural-Network-(RNN))
- [How to build your own Neural Net?](#How-to-build-your-own-Neural-Net?)
    - [Application: Predicting house prices](#Application:-Predicting-house-prices)
        - [Data](#Data)
            - [Preprocessing](#Preprocessing)
            - [Creating training and test set](#Creating-training-and-test-set)
            - [Normalizing-the-data](#Normalizing-the-data)
        - [Building a Linear Regression model with PyTorch](#Building-a-Linear-Regression-model-with-PyTorch)
            - [Create tensors](#Create-tensors)
            - [Define and train a model with PyTorch](#Define-and-train-a-model-with-PyTorch)
        - [Your turn](#Your-turn)
- [Implement a pre-built Neural Net](#Implement-a-pre-built-Neural-Net)

## Background

[Artificial Neural Networks](https://en.wikipedia.org/wiki/Artificial_neural_network) - simply called Neural Nets (NNs) - are computing systems inspired by the biological neural networks that constitute animal brains. They are used to approximate functions that are generally unknown.

NNs are based on a collection of connected nodes, called **artificial neurons**. In short, an [artificial neuron](https://en.wikipedia.org/wiki/Artificial_neuron) is a mathematical function conceived as a model of biological neurons. 

These artificial neurons are aggregated into **layers**. Signals travel from the first layer (the *input layer*), to the last layer (the *output layer*), traversing one or several *hidden layers*, which perform different transformations on their inputs. The output of one layer is the input of the next one: this is called **forward propagation**. A neural network with multiple layers between the input and output layers is called [Deep Neural Network](https://en.wikipedia.org/wiki/Deep_learning#Deep_neural_networks) (DNN).

Below is an illustration of a simple NN. Each circle represents an artificial neuron and an arrow represents a connection from the output of one artificial neuron to the input of another:

<center>
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/444px-Colored_neural_network.svg.png?20130228185515' width="270"></center>

Source: [Glosser.ca](https://commons.wikimedia.org/wiki/File:Colored_neural_network.svg), Wikipedia, [Artificial Neural Network](https://en.wikipedia.org/wiki/Artificial_neural_network)

Depending on the structure of the network, we differentiate between feedforward and recurrent NNs:
- [Feedforward Neural Network](https://en.wikipedia.org/wiki/Feedforward_neural_network) (FFNN)  was the first and simplest type of artificial neural network devised. In FFNNs, connections between the nodes do not form a cycle. FFNN are trained by backward propagation ([**backpropagation**](https://en.wikipedia.org/wiki/Backpropagation)).
- [Recurrent Neural Networks](https://en.wikipedia.org/wiki/Recurrent_neural_network) (RNN) are networks wherein connections between nodes can create a cycle, allowing output from some nodes to affect subsequent input to the same nodes. RNNs are trained by [**backpropagation through time**](https://en.wikipedia.org/wiki/Backpropagation_through_time) (BPTT).

In the following, we will explore the concept of (artificial) neurons and present a few types of neural nets.

For a nice visual introduction to Neural Nets and to backpropagation, you can view the series of videos by [3Blue1Brown](https://www.youtube.com/@3blue1brown) on the topic:

In [ ]:
YouTubeVideo("aircAruvnKk", width="560", height="315")

Further reading:
- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/), by Aurélien Géron
- [Everything you need to know about Neural Networks and Backpropagation — Machine Learning Easy and Fun](https://towardsdatascience.com/everything-you-need-to-know-about-neural-networks-and-backpropagation-machine-learning-made-easy-e5285bc2be3a), by Gavril Ognjanovski
- [Neural Network from scratch in Python](https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65), by Omar Aflak

### Neurons

#### Biological Neuron

The nervous system is composed of more than 100 billion cells known as [neurons](https://en.wikipedia.org/wiki/Neuron), which process and transmit the information received from our senses. Neurons are arranged together in our brain to form a network of nerves. These nerves pass electrical impulses from one neuron to the other.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*vBIWWCFLLzZzGes1HcCJOw.jpeg' width="450">

Neurons are made up of three major parts: 
- the *dendrites* collect information from other cells and send this information to the soma;
- the *soma* is the cell body containing the nucleus of the cell and keeping it alive;
- the *axon* transmits information away from the cell body towards other neurons or to the muscles and glands;
    - when the axon is stimulated by an electrical signal from the dendrites, the impulse is transmitted if the electrical signal is strong enough that it passes a certain level or threshold.
    - the axon terminal, located at the end of the axon farthest from the soma, contains [*synapses*](https://en.wikipedia.org/wiki/Synapse), which are structure that permits the transmission of information to another cell. Synapses have the ability to strengthen or weaken over time in responses to increases or decreases in their activity: this is called [*synaptic plasticity*](https://en.wikipedia.org/wiki/Synaptic_plasticity).

<center>
<img src='https://opentextbc.ca/introductiontopsychology/wp-content/uploads/sites/9/2013/11/6a3f0732c22683476ea201ffc5e428ad.jpg' width="400"></center>

Source: Jennifer Walinga, [The Neuron Is the Building Block of the Nervous System](https://opentextbc.ca/introductiontopsychology/chapter/3-1-the-neuron-is-the-building-block-of-the-nervous-system/), Chapter 4.1 of *Introduction to Psychology*

#### Artificial neuron and perceptron

An [artificial neuron](https://en.wikipedia.org/wiki/Artificial_neuron) is inspired by its biological counterpart. It receives some inputs, transforms them, and transmits the output.  

An artificial neuron typically: 1) performs a weighted sum of the inputs; 2) passes this sum through an [activation function](https://en.wikipedia.org/wiki/Activation_function). We'll discuss what is an activation function below. For now, let's formalize what is happening in an artificial neuron. Let: 
- $\boldsymbol{x}=(x_1, ..., x_d)$ be some input signals
    - the input signals can come from artificial neurons from the input layer (input data) or from artificial neurons from the hidden layers
- $y_k$ be the output signal of neuron $k$

The operation performed by an artificial neuron is: 
$$y_k = \varphi(\boldsymbol{w} \cdot \boldsymbol{x} + b) = \varphi(w_{k1} x_{1} +  w_{k2} x_{2} + ... +  w_{kd} x_{d} + b_k) $$

Where:
- $\boldsymbol{w}=(w_{k1}, ..., w_{kd})$ are the **weights**
    -  the weights have the same role than the synapses. Their value determines the strength of the link between one artificial neuron and another. When we learn the weights, this link can strengthen or weaken, similarly than with synaptic plasticity
- $b_k$ is the **bias**
- $\varphi$ is an **activation function**
    - the activation function models what is happening in the axon of a biological neuron: the electrical impulses are only transmitted if they are strong enough to reach a given threshold

<center>
<img src='https://www.gabormelli.com/RKB/images/thumb/3/31/artificial-neuron-model.png/600px-artificial-neuron-model.png' width="450"></center>

Source: [Artificial Neuron](https://www.gabormelli.com/RKB/Artificial_Neuron), Gabor Melli's Research Knowledge Base

Note that, oftentimes, we write $b_k=w_{k0}$ and $x_{0}=1$, so that the expression simplifies to $y_k = \varphi(\boldsymbol{w} \cdot \boldsymbol{x})$.

The first artificial neuron was the [perceptron](https://en.wikipedia.org/wiki/Perceptron), invented in 1943 by Warren McCulloch and Walter Pitts. It was a binary classifier, i.e., the activation function was the unit step function: 
- $\varphi(\boldsymbol{x}) = 1$ if $\boldsymbol{w} \cdot \boldsymbol{x} +b>0$
- $\varphi(\boldsymbol{x}) = 0$ otherwise

The perceptron, also called the single-layer perceptron, was the first and simplest (feedforward) neural network!

##### Activation function

As mentioned above, the [activation function](https://en.wikipedia.org/wiki/Activation_function) represents what is happening in the axon of a biological neuron: the electrical impulses are only transmitted if they are strong enough to reach a given threshold. There are many possible activation functions:
- unit step function ([Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function)), as used in the perceptron;
- sigmoid, e.g., the [logistic function](https://en.wikipedia.org/wiki/Logistic_function), previously seen for logistic regression
    - $\varphi(x)=\frac{1}{1 + e^{-x}}$
- Rectified linear unit ([ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))), one of the most popular because of its good performance and fast gradient computation
    - $\varphi(x) = \max(0,x) $
- Parametric rectified linear unit ([PReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)#Parametric_ReLU)), a variant of ReLU that avoids the *dead* neurons problem, i.e., neurons becoming inactive
    - $\varphi(x) = \max(\alpha x,x) $ with $0<\alpha<1$
    - when $\alpha=0.01$, the function is called Leaky ReLU

<center>
<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*ypsvQH7kvtI2BhzR2eT_Sw.png' width="400"> </center>

Source: Danqing Liu, [A Practical Guide to ReLU](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7)

There are numerous other activation functions. Explore them [here](https://en.wikipedia.org/wiki/Activation_function#Comparison_of_activation_functions)!

### Some Neural Nets


#### Multilayer Perceptron (MLP)

A [Multilayer Perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP) is a *fully-connected* feedforward neural network. Here is an illustration, with 3 hidden layers:

<center>
<img src='https://miro.medium.com/v2/resize:fit:569/0*03elfV9p5nZTnRKe.png' width="500"> </center>

Each circle represents one perceptron. Historically, the perceptrons used a sigmoid activation function, but nowadays, ReLU and its variants are more frequently used.

Originally developed in the 1960s, these models had applications in diverse fields in the 1980s, such as speech recognition and machine translation software. Recently, interest in these networks was renewed due to the successes of deep learning.  

#### Convolutional Neural Network (CNN)

A [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN) is a class of neural network most commonly applied in processing data that has a grid-like topology, such as an image. They have proven very effective in analyzing visual imagery, e.g., image recognition and classification. 

CNNs use a mathematical operation called [convolution](https://en.wikipedia.org/wiki/Convolution) in place of general matrix multiplication in at least one of their layers. This layer performs a dot product between two matrices, where one matrix is the set of learnable parameters otherwise known as a [kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing)), and the other matrix is the restricted portion of the receptive field. In other words, a convolution is the process of adding each element of the image to its local neighbors, weighted by the kernel:

<center>
<img src='https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif' width="300"> </center>

Source: Michael Plotke, [Kernel (image processing)](https://en.wikipedia.org/wiki/Kernel_(image_processing))

The kernel is a filter that applies a transformation to the original image, e.g., blurring, sharpening, embossing, edge detection, etc. 

For a more detailed introduction of CNN, you can read:
- [An Intuitive Explanation of Convolutional Neural Networks](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/), by Ujjwal Karn
- [Convolutional Neural Networks, Explained](https://towardsdatascience.com/convolutional-neural-networks-explained-9cc5188c4939), by Mayank Mishra

#### Recurrent Neural Network (RNN)

A [Recurrent Neural Networks](https://en.wikipedia.org/wiki/Recurrent_neural_network) (RNN) is a class of artificial neural networks where connections between nodes can create a cycle, allowing output from some nodes to affect subsequent input to the same nodes. RNNs are useful when dealing with sequential data like natural language and time series. They are mostly used in the fields of natural language processing and speech recognition.

A RNN is very much alike feedforward neural nerworks, except it also has connections pointing backforward: at each time step $t$, a *recurrent neuron* receives the input $x_t$ as well as its own output from the previous step $y_{t-1}$:
$$y_{t} = \varphi(\boldsymbol{w_t} \cdot \boldsymbol{x_t} + \boldsymbol{w_y} \cdot \boldsymbol{y_{t-1}} + b)$$

We can represent a recurrent neuron graphically:
<center>
<img src='https://www.oreilly.com/api/v2/epubs/9781492037354/files/assets/mlst_1401.png' width="400"> </center>


More generally, a recurrent unit may store more information than the previous predictions. A part of a neural network that preserves some state across time steps is called a *memory cell*. Indeed, since the output at time $t$ is a function of the inputs from previous time steps, the recurrent neuron has a form of memory! The memory cell is a hidden layer that flows through time:

<center>
<img src='https://www.oreilly.com/api/v2/epubs/9781492037354/files/assets/mlst_1403.png' width="400"> </center>


RNNs have the obvious advantage of taking into account historical information. However, computation can be slow and it might be difficult to access information from a long time ago (issue of *fading memory*). To solve this issue, various memory cell architectures with long-term memory have been developed to save information, including:  
- The [Long Short-Term Memory](https://en.wikipedia.org/wiki/Long_short-term_memory) (LSTM) with the idea to store a short-term state $h_t$ and a long-term state $c_t$. LTSM rely on *gates*: when the gate is open, the output is multiplied by 1 (information preserved); when the gate is closed, the outputs is multiplied by 0 (information erased). Gates are a way to update and reset persistent information:
    - the *forget gate* controls which parts of the long-term state should be erased,
    - the *input gate* controls which parts of the output should be added to the long-term state,
    - the *output gate* controls which parts should be output at this time step
- The [Gated Recurrent Unit](https://en.wikipedia.org/wiki/Gated_recurrent_unit) (GRU), a simplified version of the LTSM.

<center>
<img src='https://www.oreilly.com/api/v2/epubs/9781492037354/files/assets/mlst_1413.png' width="400"> </center>

For a more detailed introduction of RNN, you can read:
- [Source behind all the figures in this section]: [Chapter 4. Recurrent Neural Networks](https://www.oreilly.com/library/view/neural-networks-and/9781492037354/ch04.html), *Neural networks and deep learning*, by Aurélien Géron
- [Recurrent Neural Networks cheatsheet](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks), by Afshine Amidi

## How to build your own Neural Net?

There exists several libraries to implement Neural Nets in Python, the most popular being PyTorch and TensorFlow/Keras: 
- [PyTorch](https://pytorch.org/) is a deep learning framework based on [Torch](https://en.wikipedia.org/wiki/Torch_(machine_learning)). It was developed by Meta AI (Facebook) and open-sourced in 2017.
- [TensorFlow](https://www.tensorflow.org/) is an end-to-end open-source platform for machine learning. It was developed by Google and released as open source in 2015. Its name comes from the basic data structure used, namely [tensors](https://en.wikipedia.org/wiki/Tensor). 
- [Keras](https://keras.io/) is a high-level neural networks library that is running on the top of TensorFlow, among others. 

For a comparison of TensorFlow, Keras, and Pytorch, you can refer to:
- [PyTorch vs TensorFlow for Your Python Deep Learning Project](https://realpython.com/pytorch-vs-tensorflow/), by Ray Johns
- [Keras vs Tensorflow vs Pytorch: Key Differences Among Deep Learning](https://www.simplilearn.com/keras-vs-tensorflow-vs-pytorch-article), by John Terra

In this notebook, we are going to implement a Neural Net with PyTorch, displaying the simplicity, ease of use, and flexibility of this library. For a walkthrough on Neural Nets with TensorFlow/Keras, you can for instance refer to [Deep Learning with Python](https://github.com/fchollet/deep-learning-with-python-notebooks), by François Chollet (Chapter 7).

### Application: Predicting house prices

In this application, we will implement a Linear Regression model with PyTorch to predict house prices using the [Ames Housing dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

#### Data

In [ ]:
raw_data = pd.read_csv("https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/house_price.csv")
raw_data.head()

In [ ]:
raw_data.shape

The dataset contains 81 columns. A description of the features is available in the file "[house_price_data_description](https://github.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/blob/main/data/house_price_data_description.txt)"

##### Preprocessing

Let's first extract the features of interest. We will use the numeric columns:

In [ ]:
# Data types
raw_data.dtypes

In [ ]:
# Display numeric features (integer and floats)
numeric_columns = list(raw_data.columns[(raw_data.dtypes==np.int64) |
                 (raw_data.dtypes==np.float64)])
print(numeric_columns, "\n", len(numeric_columns))

`SalePrice` is the value we want to predict. We set it as the last column:

In [ ]:
# Output SalePrice as last column
numeric_columns.remove('SalePrice')
numeric_columns.append('SalePrice')

We also remove the `Id` column:

In [ ]:
# Remove Id
numeric_columns.remove('Id')

Now we extract the numeric data.

In [ ]:
# Extract numeric data
numeric_data = raw_data[numeric_columns]
numeric_data.head()

Now let's deal with the missing values in the data.

In [ ]:
# Display features with missing values
nan_columns = np.any(pd.isna(numeric_data), axis = 0)
nan_columns = list(nan_columns[nan_columns == True].index)
nan_columns

We simply replace them with zero.

In [ ]:
# Replace NAN with 0
numeric_data = numeric_data.fillna(0)

##### Creating training and test set

Let's split the data for training and test. We use the `train_test_split` module of `sklearn`:

In [ ]:
# Splitting training/test set
numeric_data_train, numeric_data_test = train_test_split(numeric_data, test_size=0.1, random_state=7)

##### Normalizing the data

Before training our linear regression model, we have to normalize the data. We do this by subtracting each column from its minimum value and then dividing it by the difference between maximum and minimum. We use the `MinMaxScaler` of `sklearn`.

In [ ]:
#Define the scaler
scaler = MinMaxScaler()
#Fit the scaler
scaler.fit(numeric_data_train)
#Transform the train and the test set
numeric_data_train.loc[:,:] = scaler.transform(numeric_data_train)
numeric_data_test.loc[:,:] = scaler.transform(numeric_data_test)

Finally, we split the column we want to predict ("SalePrice") to our features:

In [ ]:
# Extract features and output
numeric_x_columns = list(numeric_data_train.columns)
numeric_x_columns.remove("SalePrice")
X_train_df = numeric_data_train[numeric_x_columns]
y_train_df = pd.DataFrame(numeric_data_train["SalePrice"])
X_test_df = numeric_data_test[numeric_x_columns]
y_test_df = pd.DataFrame(numeric_data_test["SalePrice"])

Ok, all set, we can start building our Neural Net!

#### Building a Linear Regression model with PyTorch

We use the `PyTorch` library ([Documentation](https://pytorch.org/), imported at the beginning of this notebook with the following lines of codes:

```python
import torch
import torch.nn as nn
```

`torch.nn` contains the building blocks to build Neural Nets, e.g., the layers ([Documentation](https://pytorch.org/docs/stable/nn.html)).

##### Create tensors

The first step is to convert the data into torch tensors. A `torch.Tensor` is a multi-dimensional matrix containing elements of a single data type. It's very similar to arrays in `NumPy`.

We rely on `torch.tensor()` for the conversion ([Documentation](https://pytorch.org/docs/stable/tensors.html)).

In [ ]:
X_train = torch.tensor(X_train_df.values, dtype=torch.float)
y_train = torch.tensor(y_train_df.values, dtype=torch.float)
X_test = torch.tensor(X_test_df.values, dtype=torch.float)
y_test = torch.tensor(y_test_df.values, dtype=torch.float)

In [ ]:
print(X_train.size(), y_train.size())

##### Define and train a model with PyTorch

A model is defined as a `class` in PyTorch. Classes are a means of bundling data and functionality together, allowing to create a new type of Python object. You can read the Python documentation on [Classes](https://docs.python.org/3/tutorial/classes.html) to learn more about them.

When you create your Neural Net, you should define:
- a `__init__` function in which you define the layers of your network. 
- a `forward` function (method) that defines the forward pass on the network.

For the beginning, let's start with a single layer network:
- The layer `nn.Linear()` performs a linear transformation ([Documentation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)). The input and output are the number of neurons
- `nn.ReLU()` applies the Rectified Linear Unit function: $ReLU(x)=\max(0,x)$ ([Documentation](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)).

In [ ]:
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()
        
        self.linear1 = nn.Linear(D_in, H1)        # Linear transformation for hidden layer
        self.linear2 = nn.Linear(H1, D_out)       # Linear transformation for output layer
        self.activation = nn.ReLU()               # Activation function for hidden layer
        
    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred

`D_in` is the input dimension, i.e., the number of features. Similarly, `D_out` is the output dimension, i.e., 1 (we only predict the "SalePrice"):

In [ ]:
D_in, D_out = X_train.shape[1], y_train.shape[1]

Ok, let's define our first model. It is an instance of our newly-created class "Net". We are going to use 500 neurons for the hidden layer:

In [ ]:
# Model with 500 neurons
model1 = Net(D_in, 500, D_out)

The next steps is to define the **loss criterion** and the **optimizer** for the network. That is, we have to define the loss function we want to optimize during training and also the optimization method. We use:
- `MSELoss()` as loss criterion, i.e., the mean square error ([Documentation](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html))
- `SGD()`as optimizer, i.e., stochastic gradient descent ([Documentation](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html))

In [ ]:
# MSE loss
criterion = nn.MSELoss(reduction='sum')
# SGD optimizer for finding the weights of the network
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-4)

Wonderful, we are ready to do the training! We can simply by looping over the number of iterations. The training has 3 main steps:
- A forward pass to compute the prediction for the current data point (batch).
- Computing the loss for the current prediction with the previously defined criterion.
- A backward pass to compute the gradient of the loss with respect to the weight of the network (`backward()`)
- Finally, updating the weights of the network (`optimizer.step()`).

Note that in each backward pass PyTorch saves the gradient for all of the parameters. Therefore it is important to replace the old gradient values with zero in the beginning of each iteration (`optimizer.zero_grad()`), otherwise the gradients will be accumulated during the iterations! 

In [ ]:
losses1 = []
losses1_test = []

for t in range(500):                # 500 iterations
    
    # Forward pass: compute prediction on training set
    y_pred = model1(X_train)  
    
    # Compute loss
    loss = criterion(y_pred, y_train)
    print(t, loss.item())
    losses1.append(loss.item())
    if torch.isnan(loss):
        break
    
    # Compute gradient
    optimizer.zero_grad()    
    loss.backward()
    
    # Update 
    optimizer.step()
    
    # Compute loss on test set
    losses1_test.append(criterion(model1(X_test), y_test).item())

Let's visualize the evolution of the MSE on the training set and test set:

In [ ]:
# Plot training and test loss
plt.figure(figsize=(6, 4))
plt.plot(losses1, label="Training loss")
plt.plot(losses1_test, label="Test loss")
plt.title('Evolution of training and test loss - 500 neurons')
plt.ylim(top=70, bottom=0.0)
plt.legend()
plt.show()

Now let's try a new model with more neurons in the hidden layer. We use 1000 neurons, and follow the same steps as before:

In [ ]:
model2 = Net(D_in, 1000, D_out)

In [ ]:
# MSE loss
criterion = nn.MSELoss(reduction='sum')
# SGD optimizer for finding the weights of the network
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)

In [ ]:
losses2 = []

for t in range(500):
    y_pred = model2(X_train)
    
    loss = criterion(y_pred, y_train)
    losses2.append(loss.item())
    
    if torch.isnan(loss):
        break
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Let's visualize the evolution of the training loss for the two models:

In [ ]:
# Plot training and test loss
plt.figure(figsize=(6, 4))
plt.plot(losses1, label="Model 1: 500 neurons")
plt.plot(losses2, label="Model 2: 1000 neurons")
plt.title('Evolution of training loss')
plt.ylim(top=70, bottom=0.0)
plt.legend()
plt.show()

Let's compare the MSE loss on the test data:

In [ ]:
# prediction for model 1
model1_pred = model1(X_test)
print("MSE loss for model 1: ", criterion(model1_pred, y_test))
# prediction for model 2
model2_pred = model2(X_test)
print("MSE loss for model 2: ", criterion(model2_pred, y_test))

What do you think?

#### Your turn!

- Let's get back to model1. This time try to train it with a new optimizer. Try the Adam optimizer (which has shown to be faster than SGD for non-convex functions) and compare the training loss curve with SGD by plotting the training loss for the model trained with SGD and Adam optimizer.

Note: Use `torch.optim.Adam(model1.parameters(), lr=...)` ([Documentation](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html))

In [ ]:
# YOUR CODE HERE


- This time we want to build a new model with a new architecture. Specifically, we want to train a network with 3 hidden layers on the data. You can use the following code to build the architecture. Use the values 500, 1000, 200 for H1, H2, and H3 respectively. Train this new network on the same training data and compare it with the model1 we built above.

```
class Net_new(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(Net_new, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)
        self.activation = nn.ReLU()

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))
        y_pred = self.activation(self.linear2(y_pred))
        y_pred = self.activation(self.linear3(y_pred))
        y_pred = self.linear4(y_pred)
        return y_pred
```

In [ ]:
# YOUR CODE HERE



## Implement a pre-built Neural Net

One humongous strength of the Data Science community and of Python is the open-source values shared by its members. Hence, there exist many pre-built models for various tasks that are available for you to reuse! Using pre-trained models can reduce your computer costs, carbon footprint, and save you the time and resources required to train a model from scratch! 

[Hugging Face](https://huggingface.co/) is one of those AI community. It contains almost 200'000 [models](https://huggingface.co/models) and more than 30'000 [datasets](https://huggingface.co/datasets)!

In a few lines of codes, you can load and use pre-built models using:
- `transformers`, a library with state-of-the-art Machine Learning for PyTorch, TensorFlow and JAX. It provides thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio. Check the [Documentation](https://huggingface.co/docs/transformers/index)!
- the [Hosted Inference API](https://huggingface.co/docs/api-inference/index), an API to test and evaluate, for free, over 80'000 publicly accessible machine learning models, via simple HTTP requests

Let's try it! We will load a model using `transformers`. From this library, we import `pipeline()` (at the beginning of this notebook):

```python
from transformers import pipeline 
```

We will load the model "[bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli)", i.e., the [BART (large-sized model)](https://huggingface.co/facebook/bart-large). BART is particularly effective when fine-tuned for text generation (e.g. summarization, translation) but also works well for comprehension tasks (e.g. text classification, question answering).

The model is pre-trained on English language, and more precisely on the Multi-Genre Natural Language Inference ([MultiNLI](https://huggingface.co/datasets/multi_nli)) dataset. This corpus is a crowd-sourced collection of 433k sentence pairs annotated with textual entailment information. 

Ok, let's go! The model is loaded with the "zero-shot-classification" pipeline:

In [ ]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

[Zero-Shot Classification](https://huggingface.co/tasks/zero-shot-classification) is a task in natural language processing where a model is trained on a set of labeled examples but is then able to classify new examples from previously unseen classes.

In other words, we can provide to our classifier a sentence to classify and a list of labels, and the model with predict the class the sentence belongs to. For instance, let's say our classes are some SDGs, e.g., SDG 1 "No Poverty", SDG 5 "Gender Equality", and SDG 13 "Climate Action", and we want to classify the sentence: "In Switzerland, the greenhouse gases emissions per capita are about 14 tonnes of CO2". It's very easy:

In [ ]:
sequence_to_classify = "In Switzerland, the greenhouse gases emissions per capita are about 14 tonnes of CO2."
candidate_labels = ['No Poverty', 'Gender Equality', 'Climate Action']
classifier(sequence_to_classify, candidate_labels)

The model correctly predicted the sentence as "Climate Action".

To further improve the performance of the model for your specific task, you can **fine-tune a pre-trained model** with your own dataset in PyTorch or in TensorFlow with Keras. Here is a [tutorial](https://huggingface.co/docs/transformers/training). You can also check the post [Getting Started with Sentiment Analysis using Python](https://huggingface.co/blog/sentiment-analysis-python) by Federico Pascual for a specific guide on sentiment analysis.

Finally, as mentioned above, instead of using the `transformers` library, you can also rely on the Hosted Inference API, by making a request (you need to sign up to get an API token).

You can use the following lines of codes:

```python
import requests

API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "In Switzerland, the greenhouse gases emissions per capita are about 14 tonnes of CO2.",
    "parameters": {"candidate_labels": ['No Poverty', 'Gender Equality', 'Climate Action']},
})
```